In [ ]:
#%pip install -r requirements.txt 
from src.training_utils import * 

## **Pegasus Encoder Decoder**

In [ ]:
#import pegasus
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-large")

model_name = "google/pegasus-x-base"
model_s = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu" )
model_s.to(device)


ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)
inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model_s.generate(inputs["input_ids"])
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
"California's largest electricity provider has turned off power to hundreds of thousands of customers."


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-base")

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-x-base")

In [ ]:
input_text = "Studies have shown that owning a dog has numerous benefits."

# Tokenize the input text
input_tokens = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

output_tokens = model.generate(
    input_tokens.input_ids,
    decoder_start_token_id=model.config.pad_token_id,
    max_length=50,  # Set the desired maximum length of the generated output
    num_beams=1,  # Number of beams for beam search
    early_stopping=True,  # Stop generation when all beams have reached the end token
)

# Decode the generated output tokens
decoded_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(decoded_output)

In [ ]:

# Example input text
input_text = ["Studies have shown that owning a dog has numerous benefits.",
              "hi how are you?"]

# Tokenize the input text
input_tokens = tokenizer(input_text, return_tensors="pt",
                         truncation=True, max_length=100,
                         padding='max_length'
                         )
print(input_tokens['input_ids'].shape)
# Pass the input through the encoder
encoder_outputs = model.model.encoder(**input_tokens)

# # Access the encoder outputs
# encoder_last_hidden_state = encoder_outputs.last_hidden_state
# print(encoder_last_hidden_state.shape)